In [1]:
from magma import *
main = DefineCircuit('main', "a", In(Bit), "b", In(Bit), "c", In(Bit), "d", Out(Bit))
d = (main.a & main.b) ^ main.c
wire(d, main.d)
compile("main", main, output="firrtl")

with open("main.fir", "r") as f:
    print(f.read())

import magma
compiling And1
compiling Xor1
compiling main
circuit main :
  module And1 :
    input I0 : UInt<1>
    input I1 : UInt<1>
    output O : UInt<1>
    
    O <= and(I0, I1)
  module Xor1 :
    input I0 : UInt<1>
    input I1 : UInt<1>
    output O : UInt<1>
    
    O <= xor(I0, I1)
  module main :
    input a : UInt<1>
    input b : UInt<1>
    input c : UInt<1>
    output d : UInt<1>
    
    wire inst0_O : UInt<1>
    wire inst1_O : UInt<1>
    inst inst0 of And1
    inst0.I0 <= a
    inst0.I1 <= b
    inst0_O <= inst0.O
    inst inst1 of Xor1
    inst1.I0 <= inst0_O
    inst1.I1 <= c
    inst1_O <= inst1.O
    d <= inst1_O



In [2]:
!firrtl -i main.fir -o main.v -X verilog

Total FIRRTL Compile Time: 331.8 ms


In [3]:
with open("main.v", "r") as f:
    print(f.read())

`ifdef RANDOMIZE_GARBAGE_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_INVALID_ASSIGN
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_REG_INIT
`define RANDOMIZE
`endif
`ifdef RANDOMIZE_MEM_INIT
`define RANDOMIZE
`endif

module And1(
  input   I0,
  input   I1,
  output  O
);
  assign O = I0 & I1;
endmodule
module Xor1(
  input   I0,
  input   I1,
  output  O
);
  assign O = I0 ^ I1;
endmodule
module main(
  input   a,
  input   b,
  input   c,
  output  d
);
  wire  inst0_O;
  wire  inst1_O;
  wire  inst0__I0;
  wire  inst0__I1;
  wire  inst0__O;
  wire  inst1__I0;
  wire  inst1__I1;
  wire  inst1__O;
  And1 inst0_ (
    .I0(inst0__I0),
    .I1(inst0__I1),
    .O(inst0__O)
  );
  Xor1 inst1_ (
    .I0(inst1__I0),
    .I1(inst1__I1),
    .O(inst1__O)
  );
  assign d = inst1_O;
  assign inst0_O = inst0__O;
  assign inst1_O = inst1__O;
  assign inst0__I0 = a;
  assign inst0__I1 = b;
  assign inst1__I0 = inst0_O;
  assign inst1__I1 = c;
endmodule



In [5]:
with open("sim_main.cpp", "w") as sim_main_f:
    sim_main_f.write("""
#include "Vmain.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

int main(int argc, char **argv, char **env) {
    Verilated::commandArgs(argc, argv);
    Vmain* top = new Vmain;
    int tests[8][4] = {
        {0, 0, 0, 0},
        {1, 0, 0, 0},
        {0, 1, 0, 0},
        {1, 1, 0, 1},
        {0, 0, 1, 1},
        {1, 0, 1, 1},
        {0, 1, 1, 1},
        {1, 1, 1, 1},
    };
    for(int i = 0; i < 8; i++) {
        int* test = tests[i];
        int a = test[0];
        int b = test[1];
        int c = test[2];
        int d = test[3];

        top->a = a;
        top->b = b;
        top->c = c;

        top->eval();
        assert(top->d == ((a & b) ^ c));
    }

    delete top;
    std::cout << "Success" << std::endl;
    exit(0);
}    
""")

In [6]:
%%bash
verilator -Wall -Wno-DECLFILENAME --cc main.v --exe sim_main.cpp
make -C obj_dir -j -f Vmain.mk Vmain
./obj_dir/Vmain

clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.900/share/verilator/include -I/usr/local/Cellar/verilator/3.900/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -fbracket-depth=4096 -Qunused-arguments       -c -o sim_main.o ../sim_main.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/3.900/share/verilator/include -I/usr/local/Cellar/verilator/3.900/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -Wno-char-subscripts -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -fbracket-depth=4096 -Qunused-arguments       -c -o verilated.o /usr/local/Cellar/verilator/3.900/share/verilator/include/verilated.cpp
/usr/bin/perl /usr/local/Cellar/verilator/3.900/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include Vmain.cpp >

ar: creating archive Vmain__ALL.a
